In [54]:
#imports
import torch

from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score

In [55]:
#definim transformarea imaginilor
img_size = 224

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),  # Resize images
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize (ResNet standard)
])

In [56]:
#definim directoarele:
train_dir = "D:\Facultate\Licenta\DatasetTrim\Train"
valid_dir = "D:\Facultate\Licenta\DatasetTrim\Validation"
test_dir = "D:\Facultate\Licenta\DatasetTrim\Test"

In [57]:
train_dataset = ImageFolder(root=train_dir, transform=transform)
valid_dataset = ImageFolder(root=valid_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

In [58]:
#Creating DataLoaders - efficiently load batches of images during training.
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


In [59]:
#Load Pretrained ResNet Model
model = models.resnet50(pretrained=True)

# Modify the classifier for binary classification
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)

#Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

c:\Users\ALex\anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\ALex\anaconda3\envs\myenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [60]:
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [61]:
#Freeze pretrained parameters:
for param in model.parameters():
    param.requires_grad = False

for param in model.fc.parameters():
    param.requires_grad = True  # Train only the last layer

In [ ]:
writer = SummaryWriter()
#log_dir="D:\Facultate\Licenta\ResNet_Detection\runs"
num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        writer.add_scalar("Loss/Train", loss, epoch)
        writer.flush()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

writer.close()

Epoch 1/25, Loss: 0.6874
Epoch 2/25, Loss: 0.6498
Epoch 3/25, Loss: 0.6227
Epoch 4/25, Loss: 0.5929
Epoch 5/25, Loss: 0.5696
Epoch 6/25, Loss: 0.5495
Epoch 7/25, Loss: 0.5308
Epoch 8/25, Loss: 0.5157
Epoch 9/25, Loss: 0.4985
Epoch 10/25, Loss: 0.4892
Epoch 11/25, Loss: 0.4760
Epoch 12/25, Loss: 0.4631
Epoch 13/25, Loss: 0.4513
Epoch 14/25, Loss: 0.4517
Epoch 15/25, Loss: 0.4393
Epoch 16/25, Loss: 0.4344
Epoch 17/25, Loss: 0.4214
Epoch 18/25, Loss: 0.4167
Epoch 19/25, Loss: 0.4097
Epoch 20/25, Loss: 0.3994
Epoch 21/25, Loss: 0.3972
Epoch 22/25, Loss: 0.3982
Epoch 23/25, Loss: 0.3904
Epoch 24/25, Loss: 0.3755
Epoch 25/25, Loss: 0.3718


In [63]:
#Evaluation on the Test set
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy after fine-tuning: {accuracy * 100:.2f}%")


Test Accuracy after fine-tuning: 67.77%
